# 1. Load the data

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import random

In [2]:
questions = pd.read_csv('./data/qs_2016.csv')
# answers = pd.read_csv('./data/answ.csv')
# tags = pd.read_csv('./data/Tags.csv')

In [3]:
questions = questions.drop(['Unnamed: 0'], axis=1 )
# answers = answers.drop(['Unnamed: 0'], axis=1)

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('./data/intents.json').read()
intents = json.loads(data_file)

In [5]:
# tags_list = tags.Tag.unique().tolist()

In [6]:
"""
for tag in tags_list:
    pattern_question = []
    questions_tag_ids = tags.loc[tags['Tag']==tag].Id.unique()
    mask_questions = questions["Id"].isin(questions_tag_ids)
    questions_tag = questions.loc[mask_questions]
    for index, question in questions_tag.iterrows():
        pattern_question.append(question.Title)
    a = {
    'tag': tag,
    'patterns': pattern_question,
    'responses': [""],
    'context':[""]
    }
    intents['intents'].append(a)
    print(len(intents['intents']))
"""

'\nfor tag in tags_list:\n    pattern_question = []\n    questions_tag_ids = tags.loc[tags[\'Tag\']==tag].Id.unique()\n    mask_questions = questions["Id"].isin(questions_tag_ids)\n    questions_tag = questions.loc[mask_questions]\n    for index, question in questions_tag.iterrows():\n        pattern_question.append(question.Title)\n    a = {\n    \'tag\': tag,\n    \'patterns\': pattern_question,\n    \'responses\': [""],\n    \'context\':[""]\n    }\n    intents[\'intents\'].append(a)\n    print(len(intents[\'intents\']))\n'

In [7]:
questions_sample = questions.sample(200)

In [8]:
pattern_question=[]
for index, question in questions_sample.iterrows():
        pattern_question.append(question.Title)
        a = {
        'tag': 'question',
        'patterns': pattern_question,
        'responses': [""],
        'context':[""]
        }
        intents['intents'].append(a)

# 2. Preprocess data

In [9]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

40026 documents
5 classes ['goodbye', 'greeting', 'options', 'question', 'thanks']
884 unique lemmatized words ['#', '&', "'", "''", "'access-control-allow-origin", "'bind", "'d", "'defaultindex", "'null", "'offsetwidth", "'s", '(', ')', '+', ',', '-', '-conf', '.', '.git', '.js', '.srt', '/', '///android_asset/test.html', '/public', '1.9', '10', '104', '15', '2', '2.4', '2015', '2d', '3.0', '404', '5.1', '6', '7', '90', ':', ':drawrects', ';', '<', '>', '``', 'a', 'access', 'access-control-allow-origin', 'according', 'actionresults', 'active', 'ad', 'adc', 'add', 'adding', 'address', 'adloader', 'adwords', 'afnetworking', 'after', 'afternoon', 'aggregation', 'algorithm', 'alignment', 'all', 'allocate', 'allowed', 'always', 'am', 'amazon', 'ammap', 'amount', 'an', 'and', 'android', 'angular', 'angularjs', 'another', 'any', 'anyone', 'api', 'app', 'app.get', 'appearing', 'append', 'appended', 'application', 'apply', 'apps', 'arangodb', 'arcgis', 'architecture', 'are', 'argument', 'armad

# 3. Create training and testing data

In [11]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-11-5117620404a0>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


# 4. Build the model

In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [13]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

/home/guedri/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
8006/8006 [==============================] - 14s 1ms/step - loss: 0.0095 - accuracy: 0.9990
Epoch 2/200
8006/8006 [==============================] - 11s 1ms/step - loss: 0.0048 - accuracy: 0.9994
Epoch 3/200
8006/8006 [==============================] - 11s 1ms/step - loss: 0.0037 - accuracy: 0.9994
Epoch 4/200
8006/8006 [==============================] - 11s 1ms/step - loss: 0.0026 - accuracy: 0.9994
Epoch 5/200
8006/8006 [==============================] - 11s 1ms/step - loss: 0.0020 - accuracy: 0.9994
Epoch 6/200
8006/8006 [==============================] - 11s 1ms/step - loss: 0.0015 - accuracy: 0.9994
Epoch 7/200
8006/8006 [==============================] - 11s 1ms/step - loss: 0.0012 - accuracy: 0.9994
Epoch 8/200
8006/8006 [==============================] - 11s 1ms/step - loss: 0.0012 - accuracy: 0.9994
Epoch 9/200
8006/8006 [==============================] - 11s 1ms/step - loss: 0.0011 - accuracy: 0.9996
Epoch 10/200
8006/8006 [==============================] - 11s 1m

8006/8006 [==============================] - 11s 1ms/step - loss: 1.4525e-04 - accuracy: 1.0000
Epoch 77/200
8006/8006 [==============================] - 11s 1ms/step - loss: 1.7451e-04 - accuracy: 1.0000
Epoch 78/200
8006/8006 [==============================] - 11s 1ms/step - loss: 1.4323e-04 - accuracy: 1.0000
Epoch 79/200
8006/8006 [==============================] - 11s 1ms/step - loss: 2.1222e-04 - accuracy: 1.0000
Epoch 80/200
8006/8006 [==============================] - 10s 1ms/step - loss: 1.1202e-04 - accuracy: 1.0000
Epoch 81/200
8006/8006 [==============================] - 10s 1ms/step - loss: 9.7964e-05 - accuracy: 1.0000
Epoch 82/200
8006/8006 [==============================] - 10s 1ms/step - loss: 8.4578e-05 - accuracy: 1.0000
Epoch 83/200
8006/8006 [==============================] - 10s 1ms/step - loss: 1.5131e-04 - accuracy: 1.0000
Epoch 84/200
8006/8006 [==============================] - 10s 1ms/step - loss: 9.8626e-05 - accuracy: 1.0000
Epoch 85/200
8006/8006 [========

8006/8006 [==============================] - 11s 1ms/step - loss: 7.5030e-05 - accuracy: 1.0000
Epoch 151/200
8006/8006 [==============================] - 11s 1ms/step - loss: 7.4027e-05 - accuracy: 1.0000
Epoch 152/200
8006/8006 [==============================] - 11s 1ms/step - loss: 4.5123e-05 - accuracy: 1.0000
Epoch 153/200
8006/8006 [==============================] - 11s 1ms/step - loss: 4.6796e-05 - accuracy: 1.0000
Epoch 154/200
8006/8006 [==============================] - 11s 1ms/step - loss: 3.1601e-05 - accuracy: 1.0000
Epoch 155/200
8006/8006 [==============================] - 11s 1ms/step - loss: 2.9927e-05 - accuracy: 1.0000
Epoch 156/200
8006/8006 [==============================] - 11s 1ms/step - loss: 5.9313e-05 - accuracy: 1.0000
Epoch 157/200
8006/8006 [==============================] - 13s 2ms/step - loss: 1.2720e-04 - accuracy: 1.0000
Epoch 158/200
8006/8006 [==============================] - 13s 2ms/step - loss: 5.7632e-05 - accuracy: 1.0000
Epoch 159/200
8006/8006 

In [15]:
with open('intents.json', 'w') as fp:
    json.dump(intents, fp)